#앙상블 학습
+ 정형 데이터에 잘 맞는다. (비정형 데이터 -> 신경망 알고리즘)


### 랜덤 포레스트
+ 부트스트랩 샘플 : 중복을 허용하여 데이터를 랜덤하게 추출한 샘플 

&emsp;&emsp; (보통 훈련세트와 크기가 같다)
+ 노드를 분할할 때 전체 특성 중 일부를 무작위로 고르고 그 중에서 최선의 분할을 찾는다.

결국 여러가지의 훈련세트를 만들어서 결정트리를 여러개 훈련시킨다.

분류의 경우 각 클래스별 확률을 평균을 구해 가장 높은 확률의 클래스로 예측한다.

화귀의 경우 단순히 각 트리의 예측을 평균한다.


In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

wine = pd.read_csv('https://bit.ly/wine-date')
data = wine.drop('class', axis = 1).to_numpy()
target = wine['class'].to_numpy()
train_input, test_input, train_target, test_target = train_test_split(data, target, test_size = 0.2, random_state = 42)

In [5]:
from sklearn.model_selection import cross_validate
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier(n_jobs = -1, random_state = 42)
score = cross_validate(rf, train_input, train_target, return_train_score = True, n_jobs = -1)
print(np.mean(score['train_score']), np.mean(score['test_score']))

0.9973541965122431 0.8905151032797809


In [8]:
rf.fit(train_input, train_target)
print(rf.feature_importances_)

[0.23167441 0.50039841 0.26792718]


In [9]:
# 부트스트랩 샘플에 들어가지 않는 샘플들을 OOB샘플이라고 하며, 이 샘플들을 가지고 검증 세트처럼 이용할 수 있다
rf = RandomForestClassifier(oob_score = True, n_jobs = -1, random_state = 42)
rf.fit(train_input, train_target)
print(rf.oob_score_)

0.8934000384837406


# 엑스트라 트리

In [12]:
#엑스트라 트리는 기본 100개의 결정트리를 훈련한다. -> 부트스트랩 샘플을 사용하지 않고, 모든 샘플을 사용
# 대신 노드를 분할할 때 사용되는 특성을 랜덤으로 결정한다.
from sklearn.ensemble import ExtraTreesClassifier
et = ExtraTreesClassifier(random_state = 42, n_jobs = -1)
scores= cross_validate(et, train_input, train_target, return_train_score = True, n_jobs = -1)
print(np.mean(score['train_score']), np.mean(score['test_score']))

0.9973541965122431 0.8905151032797809


In [14]:
et.fit(train_input, train_target)
print(et.feature_importances_)

[0.20183568 0.52242907 0.27573525]


# 그레이디언트 부스팅
+ 깊이가 낮은( 기본적으로는 3) 트리 100개를 훈련시킴

과대적합이 일어나기 쉽지 않고, 일반화 성능이 좋음

+ 순서대로 트리를 추가하기 때문에 n_jobs 매개변수가 없다.

In [15]:
from sklearn.ensemble import GradientBoostingClassifier
gb = GradientBoostingClassifier( random_state = 42)
score = cross_validate(gb, train_input, train_target, return_train_score = True, n_jobs = -1)
print(np.mean(score['train_score']), np.mean(score['test_score']))

0.8881086892152563 0.8720430147331015


In [17]:
#과대적합이 거의 없다. 학습률을 증가시켜 더 좋은 성능을 기대해보자
gb = GradientBoostingClassifier(n_estimators = 500,learning_rate = 0.2, random_state = 42)
score = cross_validate(gb, train_input, train_target, return_train_score = True, n_jobs = -1)
print(np.mean(score['test_score']), np.mean(score['test_score']))

0.8780082549788999 0.8780082549788999


In [19]:
gb.fit(train_input, train_target)
print(gb.feature_importances_)

[0.15872278 0.68010884 0.16116839]


#히스토그램 기반 그레디언트 부스팅
+ 입력 특성을 256개 구간으로 나눈다 -> 최적의 분할을 빠르게 찾을 수 있음
+ 구간의 값중 하나를 누락된 값을 위해 사용한다. -> 전처리가 필요없음

In [23]:
from sklearn.experimental import enable_hist_gradient_boosting
from sklearn.ensemble import HistGradientBoostingClassifier
hgb = HistGradientBoostingClassifier(random_state = 42)
scores = cross_validate(hgb, train_input, train_target, return_train_score = True)
print(np.mean(score['train_score']), np.mean(score['test_score']))

0.9464595437171814 0.8780082549788999


In [26]:
#특성을 랜덤하게 섞어 모델의 성능이 변화하는지를 관찰해 특성의 중요도를 계산함
from sklearn.inspection import permutation_importance

hgb.fit(train_input, train_target)
result = permutation_importance(hgb, train_input, train_target, n_repeats = 10, random_state = 42, n_jobs = -1)
print(result.importances_mean)

[0.08876275 0.23438522 0.08027708]


In [27]:
result = permutation_importance(hgb, test_input, test_target, n_repeats = 10, random_state = 42, n_jobs = -1)
print(result.importances_mean)

[0.05969231 0.20238462 0.049     ]


In [28]:
hgb.score(test_input, test_target)

0.8723076923076923

In [29]:
# 사이킷런 이외의 곳에서 만들 히스토그램 기반 그래디언트 부스팅 모델
#1. XGB
from xgboost import XGBClassifier
xgb = XGBClassifier(tree_method = 'hist', random_state = 42)
scores = cross_validate(xgb, train_input, train_target, return_train_score = True)
print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.8824322471423747 0.8726214185237284


In [32]:
#2. LightGBM
from lightgbm import LGBMClassifier
lgb = LGBMClassifier(random_state = 42)
score = cross_validate(lgb, train_input, train_target, return_train_score = 42)
print(np.mean(score['train_score']), np.mean(score['test_score']))

0.9338079582727165 0.8789710890649293
